In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import urllib.parse

In [5]:
df = pd.read_csv('Water_Consumption_And_Cost__2013_-_Feb_2025__20250513-Copy1.csv')
df['Service Start Date'] = pd.to_datetime(df['Service Start Date'], errors='coerce')
df['Service End Date'] = pd.to_datetime(df['Service End Date'], errors='coerce')
df['Year'] = df['Service Start Date'].dt.year
df['Month'] = df['Service Start Date'].dt.month
df['Days_in_Period'] = (df['Service End Date'] - df['Service Start Date']).dt.days


In [7]:
df = df.rename(columns={
    'Development Name': 'Development',
    'Consumption (HCF)': 'Consumption_HCF',
    'Current Charges': 'Charges_USD',
    'Water&Sewer Charges': 'Water_Sewer_Charges'
})

In [9]:
for col in ['Consumption_HCF', 'Charges_USD', 'Water_Sewer_Charges']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(subset=['Consumption_HCF', 'Charges_USD', 'Service Start Date'], inplace=True)

df['Cost_Per_HCF'] = df['Charges_USD'] / df['Consumption_HCF']
df['Daily_Consumption'] = df['Consumption_HCF'] / df['Days_in_Period']

In [22]:
# Save cleaned data (optional)
df.to_csv('Cleaned_NYC_Water_Consumption.csv-copy1', index=False)

# --- Database upload ---
username = 'root'
password = urllib.parse.quote_plus('Nihil@mysql1')
host = 'localhost'
port = '3306'
db_name = 'water_usage_nyc1'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{db_name}")

# Replace inf, -inf with NaN, then drop all rows with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# Upload to MySQL
df.to_sql('water_consumption', con=engine, if_exists='replace', index=False)


14599

In [24]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE OR REPLACE VIEW vw_monthly_consumption AS
        SELECT 
            Year, Month, Borough,
            SUM(Consumption_HCF) AS Total_Consumption,
            SUM(Charges_USD) AS Total_Charges,
            SUM(Charges_USD) / SUM(Consumption_HCF) AS Cost_Per_HCF
        FROM water_consumption
        GROUP BY Year, Month, Borough
        ORDER BY Year, Month, Borough
    """))